In [1]:
%cd ..
%pwd

/home/markus/Documents/regression-prediction


'/home/markus/Documents/regression-prediction'

In [2]:
%load_ext autoreload
%autoreload 2

In [ ]:
import requests
import time
url = "https://treeherder.mozilla.org/api/performance/alertsummary/"
alertsummaries = []
i = 0
while True:
    i += 1
    print(f"{i}. GET {url}", end="\r")
    r = requests.get(url, headers={'User-Agent': 'basic'})
    j = r.json()
    alertsummaries += j['results']
    url = j['next']
    if url is None:
        break
    time.sleep(0.5)

In [ ]:
import datetime
date = datetime.datetime.now().strftime('%Y-%m-%d')
write_json_to_file(alertsummaries, f'data/labeling/alerts_{date}.json')

In [64]:
from src.utils import *
from src.labeling import *
from src.repo_miner import get_commit_log

In [89]:
selected_commits = get_selected_commits()
len(selected_commits) # 92188

Get commits: 581679it [01:27, 6612.35it/s] 


99705

In [4]:
alertsummaries = read_data_from_json('data/labeling/alerts_2022-03-01.json')
alertsummaries2 = read_data_from_json('data/labeling/alerts_2021-09-13.json')

In [5]:
alert_ids = {alert['id'] for alert in alertsummaries}
alert_ids2 = {alert['id'] for alert in alertsummaries2}

In [6]:
len(alert_ids), len(alert_ids2), len(alert_ids.intersection(alert_ids2))

(4592, 4460, 2516)

In [7]:
for alert in alertsummaries2:
    if alert['id'] not in alert_ids:
        alertsummaries.append(alert)

In [65]:
all_commits = get_all_commits()
hg_to_git, git_to_hg = get_hg_git_mapping()
commit_log = get_commit_log('data/repo_miner/commit_log.csv')

Get selected commits: 581679it [01:32, 6315.15it/s] 


Get hg to git mapping ... Done.


In [9]:
szz_labeling = pd.read_csv(f'data/labeling/fixed_defect_szz.csv')
fix_bug_ids_by_kind, _ = get_defects_and_fixes()
fix_and_introducers = read_data_from_json(f'data/labeling/fixed_defect_szz/results/fix_and_introducers_pairs.json')

Get defects and fixes: 206038it [00:31, 6533.75it/s] 

Found
46417 regression
1337 security
961 performance
122 memory
3348 crash
39 power
fix bug ids.


In [24]:
for pair in fix_and_introducers:
    pair[0] = git_to_hg[pair[0]]
    pair[1] = git_to_hg[pair[1]]

In [26]:
len(fix_and_introducers)

2495

In [66]:
fix_and_introducers_df = pd.DataFrame(fix_and_introducers, columns=['fix', 'introducer'])
fix_and_introducers_df

,fix,introducer
0,b6113f461cb8bc50be357b7e84793c11f7a5c179,a6da1179a589e207da62749f5966f30024f7f0c9
1,714d667c7f120e7218ca08fb0f29508f4c81cc92,21ce1e95d697cbc1bdaa45d124e55642ff5b2fd3
2,714d667c7f120e7218ca08fb0f29508f4c81cc92,6daa67c0cae427dc8e514d5f1beea9f71ee4d8ef
3,714d667c7f120e7218ca08fb0f29508f4c81cc92,e53cd28c6cf360d7d9aa3a35051582112e608df1
4,714d667c7f120e7218ca08fb0f29508f4c81cc92,be7a5a989cb3dbabce6a1bb78aad2d05baa3589a
...,...,...
2490,8a0c0ad101aba2f397fa58932315004aa33de2d9,1ddc47f7370187b3a07d61a65f9f0fd19e7e20a5
2491,8a0c0ad101aba2f397fa58932315004aa33de2d9,1ddc47f7370187b3a07d61a65f9f0fd19e7e20a5
2492,e3d87330819774902687ba23d1a76b5f46daf26d,1ddc47f7370187b3a07d61a65f9f0fd19e7e20a5
2493,40b67d5f6d17f1e206d283d20bc35da5272fee38,1ddc47f7370187b3a07d61a65f9f0fd19e7e20a5


In [67]:
fix_and_introducers_df = fix_and_introducers_df.merge(all_commits[['revision', 'bug_id']], left_on='fix', right_on='revision').drop('revision', axis=1)
fix_and_introducers_df

,fix,introducer,bug_id
0,b6113f461cb8bc50be357b7e84793c11f7a5c179,a6da1179a589e207da62749f5966f30024f7f0c9,1693027
1,714d667c7f120e7218ca08fb0f29508f4c81cc92,21ce1e95d697cbc1bdaa45d124e55642ff5b2fd3,1582797
2,714d667c7f120e7218ca08fb0f29508f4c81cc92,6daa67c0cae427dc8e514d5f1beea9f71ee4d8ef,1582797
3,714d667c7f120e7218ca08fb0f29508f4c81cc92,e53cd28c6cf360d7d9aa3a35051582112e608df1,1582797
4,714d667c7f120e7218ca08fb0f29508f4c81cc92,be7a5a989cb3dbabce6a1bb78aad2d05baa3589a,1582797
...,...,...,...
2490,59a0ddad5c28302d22089be78e59f2a058898842,17190bff616b3cbaca692825c64e6650a1de9562,1681030
2491,a5e88ddf8867e24f09fbb0bf0491d107cf68028d,b4197fcbd41190d40e4eec3ab4c2e270d80c7424,1665196
2492,e3d87330819774902687ba23d1a76b5f46daf26d,1ddc47f7370187b3a07d61a65f9f0fd19e7e20a5,1596709
2493,40b67d5f6d17f1e206d283d20bc35da5272fee38,1ddc47f7370187b3a07d61a65f9f0fd19e7e20a5,1596709


In [68]:
# fix_bug_ids_by_kind['performance'] are bug numbers for which we tried to find an introducer
alertsummaries_considered = [alert for alert in alertsummaries if alert['bug_number'] in fix_bug_ids_by_kind['performance']]

# total number of bug ids for which an alert exists and for which we tried to find an introducer
N = len({alert['bug_number'] for alert in alertsummaries_considered})

len(alertsummaries_considered), N

(297, 192)

In [69]:
# bug ids for which we actually found an introducer
bug_ids_found = set(fix_and_introducers_df['bug_id'])
assert len(bug_ids_found.difference(set(fix_bug_ids_by_kind['performance']))) == 0

# corresponding alerts
alertsummaries_found = [alert for alert in alertsummaries_considered if alert['bug_number'] in bug_ids_found]

# all bug ids for which an alert exists and for which we found an introducer
alert_bug_ids = {alert['bug_number'] for alert in alertsummaries_found}

TP_plus_FP = len(alert_bug_ids)

len(alertsummaries_found), TP_plus_FP

(144, 87)

In [70]:
# number of bug ids for which we tried to find an introducer but did not
# lower bound on false negative *commits*
N - TP_plus_FP

105

In [78]:
TP = 0
FP = 0

for bug_id in alert_bug_ids:
    introducers_for_bug = fix_and_introducers_df.loc[fix_and_introducers_df['bug_id'] == bug_id, 'introducer']
    introducers_for_bug = set(introducers_for_bug)
    assert len(introducers_for_bug) > 0
    
    all_candidate_revisions = set()
    for alert in alertsummaries_considered:
        if alert['bug_number'] == bug_id:
            try:
                if alert['prev_push_revision'] == alert['revision']:
                    # this is sometimes the case, we just take single commit
                    candidate_revisions = set(commit_log.loc[alert['prev_push_revision']:alert['revision'], 'revision'])
                else:
                    # performance change detected in interval (alert['prev_push_revision'], alert['revision']]
                    candidate_revisions = set(commit_log.loc[alert['prev_push_revision']:alert['revision'], 'revision'].iloc[1:])
            except KeyError:
                pass
            all_candidate_revisions = all_candidate_revisions.union(candidate_revisions)

    TP += len(introducers_for_bug.intersection(candidate_revisions))
    FP += len(introducers_for_bug.difference(candidate_revisions))

TP, FP, TP + FP
            

(38, 691, 729)

In [10]:
_, _, _, bugbug_fix_to_regressor = get_bugbug_regressors_and_fixes()


Get regressors and fixes: 206038it [00:29, 6899.19it/s] 


Found
9562 regression
874 performance
1214 crash
281 security
32 memory
9 power
regressor bug ids,

and

15334 regression
965 performance
1577 crash
299 security
31 memory
9 power
fix bug ids.


In [59]:
bugbug_fixes = set(bugbug_fix_to_regressor.keys())
len(bugbug_fixes)

15334

In [60]:
bug_ids_considered = bugbug_fixes.intersection(set(fix_bug_ids_by_kind['performance']))
N = len(bug_ids_considered)
N

371

In [63]:
# bug ids for which we actually found an introducer
bug_ids_found = set(fix_and_introducers_df['bug_id'])
assert len(bug_ids_found.difference(set(fix_bug_ids_by_kind['performance']))) == 0

regressed_by_bug_ids = bug_ids_considered.intersection(bug_ids_found)
TP_plus_FP = len(regressed_by_bug_ids)
TP_plus_FP

164

In [62]:
# number of bug ids for which we tried to find an introducer but did not
# lower bound on false negative *commits*
N - TP_plus_FP

207

In [77]:
TP = 0
FP = 0

for bug_id in regressed_by_bug_ids:
    introducers_for_bug = fix_and_introducers_df.loc[fix_and_introducers_df['bug_id'] == bug_id, 'introducer']
    introducers_for_bug = set(introducers_for_bug)
    assert len(introducers_for_bug) > 0
    
    all_candidate_revisions = set()
    for regressors_bug_id in bugbug_fix_to_regressor[bug_id]:
        candidate_revisions = set(all_commits.loc[all_commits['bug_id'] == regressors_bug_id, 'revision'])
        all_candidate_revisions = all_candidate_revisions.union(candidate_revisions)


    TP += len(introducers_for_bug.intersection(candidate_revisions))
    FP += len(introducers_for_bug.difference(candidate_revisions))

TP, FP, TP + FP

(134, 576, 710)

In [83]:
resolutions = set()
status = set()
types = set()
my_bug = None
with open('data/bugbug/bugs.json', encoding="utf-8") as f:
    for line in tqdm(f, desc='Get defects and fixes'):
        bug = json.loads(line)
        if bug['id'] == 1717171:
            my_bug = bug
        resolutions.add(bug['resolution'])
        status.add(bug['status'])
        types.add(bug['type'])

Get defects and fixes: 206038it [00:24, 8383.67it/s] 


In [84]:
resolutions

{'',
 'DUPLICATE',
 'EXPIRED',
 'FIXED',
 'INACTIVE',
 'INCOMPLETE',
 'INVALID',
 'MOVED',
 'WONTFIX',
 'WORKSFORME'}

In [85]:
status

{'ASSIGNED',
 'CLOSED',
 'NEW',
 'REOPENED',
 'RESOLVED',
 'UNCONFIRMED',
 'VERIFIED'}

In [86]:
types

{'--', 'defect', 'enhancement', 'task'}

In [ ]:
#labeling = 'fixed_defect_szz'

bugbug_szz_labeling = pd.read_csv(f'data/labeling/bugbug_szz.csv')
fixed_defect_szz_labeling = pd.read_csv(f'data/labeling/fixed_defect_szz.csv')
# fix_and_introducers = read_data_from_json(f'data/labeling/{labeling}/results/fix_and_introducers_pairs.json')

_, bugbug_fix_bug_ids_by_kind, _ = get_bugbug_regressors_and_fixes()

fixed_defect_fix_bug_ids_by_kind, _ = get_defects_and_fixes()


In [ ]:
bugbug_szz_labeling['performance'].sum()

In [ ]:
fixed_defect_szz_labeling['performance'].sum()

In [ ]:
import numpy as np
np.maximum(fixed_defect_szz_labeling['performance'], bugbug_szz_labeling['performance']).sum()

In [ ]:
len(fix_bug_ids_by_kind['performance'])

In [ ]:
import pandas as pd


In [ ]:
fixes = set(fix for fix, introducer in fix_and_introducers)
len(fixes)

In [ ]:
A = set(fixed_defect_fix_bug_ids_by_kind['performance'])
B = set(bugbug_fix_bug_ids_by_kind['performance'])
I = A.intersection(B)
len(I)

In [ ]:
len(A.difference(I)), len(B.difference(I))

In [ ]:
alertsummaries_filtered = [a for a in alertsummaries if a['bug_number'] in fixed_defect_fix_bug_ids_by_kind['performance']]
len(alertsummaries_filtered)

In [ ]:
alertsummaries_filtered = [a for a in alertsummaries if a['bug_number'] in bugbug_fix_bug_ids_by_kind['performance']]
len(alertsummaries_filtered)

In [ ]:
alert_bug_ids = set(a['bug_number'] for a in alertsummaries)
len(alert_bug_ids.intersection(fixed_defect_fix_bug_ids_by_kind['performance']))

In [ ]:
alert_bug_ids = set(a['bug_number'] for a in alertsummaries)
len(alert_bug_ids.intersection(bugbug_fix_bug_ids_by_kind['performance']))